In [1]:
import os, os.path as op
import glob, shutil
import argparse, sys, copy, time
import matplotlib.pyplot as plt
import numpy as np, pandas as pd
import cProfile
import pstats
from datetime import datetime

%pdb off
%matplotlib notebook

Automatic pdb calling has been turned OFF


#### Setup Structure

In [2]:
## you will need to edit this
path_AT      = op.join(op.expanduser('~'), 'ARIA-tools_git')
path_tssetup = shutil.which('ariaTSsetup.py')
assert path_tssetup, 'Cannot find ariaTSsetup.py'

In [3]:
# get the git branch
os.chdir(path_AT)
gbranch = !git branch --show-current
gbranch = gbranch[0]
print ('On branch:', gbranch)

On branch: Profiling


In [4]:
# setup directory for aria outputs and profiles
track   = '004'

path_wd = op.join(op.expanduser('~'), f'Virtual_Tests_{track}') 
os.makedirs(path_wd, exist_ok=True)
os.chdir(path_wd)

path_prods = op.join(path_wd, 'products')
os.makedirs(path_prods, exist_ok=True)

path_profs = op.join(path_wd, 'Profiles')
os.makedirs(path_profs, exist_ok=True)

print ('Working directory:', path_wd)

Working directory: /home/jovyan/Virtual_Tests_004


In [5]:
bbox      = '36.75 37.225 -76.655 -75.928' # Hampton Roads
n_threads = 4

In [6]:
def clear_directory(path):
    """ Start from a clean directory """
    files = os.listdir(path)
    # keep the DEM as it saves some time
    files = [op.join(path, f) for f in files if not 'DEM' in f]
    !rm -rf {' '.join(files)}
    return

## Single Frame

In [7]:
# 18 products between these dates for a single frame
st, en, nprod = 20170201, 20170501, 18 
# st, en, nprod = 20170101, 20180101, 80 ## longer runs

path_proc = op.join(path_wd, f'1Frame_{nprod}_{gbranch}')
path_dem  = op.join(path_proc, 'DEM', 'SRTM_3arcsec.dem')

os.makedirs(path_proc, exist_ok=True)
os.chdir(path_proc)

In [8]:
# start from a fresh directory
clear_directory(path_proc)
clear_directory(path_prods)

print ('Ready to download ARIA to:', path_proc)

Ready to download ARIA to: /home/jovyan/Virtual_Tests_004/1Frame_18_Profiling


### Download

In [9]:
# download the physical products; 
# really this should count towards the time vs for virtual profiling, but it's pretty fast and makes comparison overly difficult
cmd = f'ariaDownload.py -t {track} --st {st} --en {en}  --b "{bbox}" -w {path_prods} -nt 4'
print (cmd)
!{cmd}

ariaDownload.py -t 004 --st 20170201 --en 20170501  --b "36.75 37.225 -76.655 -75.928" -w /home/jovyan/Virtual_Tests_004/products -nt 4
ARIA-tools Version: 1.1.0
Wrote -- 18 -- products to: /home/jovyan/Virtual_Tests_004/products


In [10]:
# download the text file with the virtual urls
cmd = f'ariaDownload.py -t 004 --st {st} --en {en}  --b "{bbox}" -w {path_prods} -nt 4 -o url'
print (cmd)
!{cmd}

ariaDownload.py -t 004 --st 20170201 --en 20170501  --b "36.75 37.225 -76.655 -75.928" -w /home/jovyan/Virtual_Tests_004/products -nt 4 -o url
ARIA-tools Version: 1.1.0
Wrote -- 18 -- product urls to: /home/jovyan/Virtual_Tests_004/products/track004_bbox-77W36S-75E38N_0.txt


In [11]:
cmd = f'ariaExtract.py -f "{path_prods}/*.nc" --dem Download -nt 4'
print (cmd)
!{cmd}

ariaExtract.py -f "/home/jovyan/Virtual_Tests_004/products/*.nc" --dem Download -nt 4
ARIA-tools Version: 1.1.0
*****************************************************************
*** Extract Product Function ***
*****************************************************************
Multi-core version
All (18) GUNW products meet spatial bbox criteria.
Group GUNW products into spatiotemporally continuous interferograms.
All (18) interferograms are spatially continuous.
No layers specified; only creating bounding box shapes
Thread count specified for gdal multiprocessing = 4
Applied cutline to produce 3 arc-sec SRTM DEM: ./DEM/SRTM_3arcsec.dem


### tsSetup1 - Physical

In [12]:
n     = 2 # run it twice; stats are stable

# setup the file name of the profile
today = datetime.today().strftime('%m%d') 
bname = f'PHYS_{nprod}_{today}'

In [ ]:
for nn in range(n):
    dst = op.join(path_profs, f'{bname}_{nn}.profile')
                
    cmd = f'python -m cProfile -o {dst} '\
          f'-s time {path_tssetup} -f "{path_prods}/S1*.nc" '\
          f'--dem {path_dem} --layers "all" --bperp -nt {n_threads} '
    
    print (cmd)
    !{cmd}
    print ('\n-------Finished:', dst)
    clear_directory(path_proc)

python -m cProfile -o /home/jovyan/Virtual_Tests_004/Profiles/PHYS_18_0417_0.profile -s time /home/jovyan/.local/envs/ARIA/bin/ariaTSsetup.py -f "/home/jovyan/Virtual_Tests_004/products/S1*.nc" --dem /home/jovyan/Virtual_Tests_004/1Frame_18_Profiling/DEM/SRTM_3arcsec.dem --layers "all" --bperp -nt 4 
ARIA-tools Version: 1.1.0
*****************************************************************
*** Time-series Preparation Function ***
*****************************************************************
Multi-core version
All (18) GUNW products meet spatial bbox criteria.
Group GUNW products into spatiotemporally continuous interferograms.
All (18) interferograms are spatially continuous.
Thread count specified for gdal multiprocessing = 4
Download/cropping DEM

Extracting unwrapped phase, coherence, and connected components for each interferogram pair
Generating: unwrappedPhase - [==>                      6%                       ] 20170215_20170203    0s /     0sNETCDF:"/home/jovyan/Virtual_

### tsSetup1 - Virtual

In [ ]:
n     = 2 # run it n times

# setup the file name of the profile
today = datetime.today().strftime('%m%d') 
bname = f'VIRT_{nprod}_{gbranch}_{today}'

In [ ]:
for nn in range(n):
    dst = op.join(path_profs, f'{bname}_{nn}.profile')
                
    cmd = f'python -m cProfile -o {dst} '\
          f'-s time {path_tssetup} -f "{path_prods}/track004_bbox-77W36S-75E38N_0.txt" '\
          f'--dem {path_dem} --layers "all" --bperp -nt {n_threads} '
    print (cmd)
    !{cmd}
    print ('\n-------Finished:', dst)
    clear_directory(path_proc)

## Multiple Frames

In [ ]:
# 21 products between these dates for multiple frames
st_tr, en_tr, nprod_tr = 20190101, 20190220, 21
# st_tr, en_tr, nprod_tr  = 20170101, 20170315, 83 

path_proc = op.join(path_wd, f'Track_{nprod_tr}_{gbranch}')
os.makedirs(path_proc, exist_ok=True)
os.chdir(path_proc)

In [ ]:
# start from a fresh directory
clear_directory(path_res)

print ('Ready to download ARIA to:', path_proc)

### Download

In [ ]:
# download the physical products; 
# really this should count towards the time vs for virtual profiling, but it's pretty fast and makes comparison overly difficult
cmd = f'ariaDownload.py -t {track} --st {st_tr} --en {en_tr} -nt 4'
print (cmd)
!{cmd}

In [ ]:
# download the text file with the virtual urls
cmd = f'ariaDownload.py -t 004 --st {st_tr} --en {en_tr} -nt 4 -o url'
print (cmd)
!{cmd}

In [ ]:
cmd = f'ariaExtract.py -f "{path_proc}/products/*.nc" --dem Download -nt 4'
print (cmd)
!{cmd}

### tsSetup+ - Physical

In [ ]:
n     = 2 # run it twice; stats are stable

# setup the file name of the profile
today = datetime.today().strftime('%m%d') 
bname = f'PHYS_{nprod_tr}_{gbranch}_{today}'

In [ ]:
for nn in range(n):
    dst = op.join(path_profs, f'{bname}{nn}.profile')
                
    cmd = f'python -m cProfile -o {dst} '\
          f'-s time {path_tssetup} -f "{path_proc}/products/S1*.nc" '\
          f'--dem {path_dem} --layers "all" --bperp -nt {n_threads} '
    print (cmd)
    !{cmd}
    print ('\n-------Finished:', dst)
    clear_directory(path_proc)

### tsSetup+ - Virtual

In [ ]:
n     = 5 # run it nx

# setup the file name of the profile
today = datetime.today().strftime('%m%d') 
bname = f'VIRT_{nprod_tr}_{gbranch}_{today}'

In [ ]:
for nn in range(n):
    dst = op.join(path_profs, f'{bname}{nn}.profile')
                
    cmd = f'python -m cProfile -o {dst} '\
          f'-s time {path_tssetup} -f "{path_proc}/products/track004_0.txt" '\
          f'--dem {path_dem} --layers "all" --bperp -nt {n_threads} '
    print (cmd)
    !{cmd}
    print ('\n-------Finished:', dst)
    clean_dir(path_proc)